In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gsm8k-dataset/test.jsonl
/kaggle/input/gsm8k-dataset/train.jsonl
/kaggle/input/gsm8k-dataset/test_prompts.csv


In [2]:
import torch

In [3]:
# Set up the GPU device
device = torch.device("cuda")

# Downloading the model

In [4]:
import torch
from transformers import BloomTokenizerFast, BloomForCausalLM

tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-1b7")
model = BloomForCausalLM.from_pretrained("bigscience/bloom-1b7")

In [9]:
model = model.to(device)

# Testing it on our prompt

In [83]:
import pandas as pd
df = pd.read_csv("/kaggle/input/gsm8k-dataset/test_prompts.csv")
text = df.loc[0, "CoT_anot1"]

Looking at the text firstly

In [84]:
text

"Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nA: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.\nQ: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\nA: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.\nQ: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\nA: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.\nQ: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\nA: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denn

In [96]:
inputs = tokenizer.encode(text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128, num_return_sequences=4, num_beams=4)

The model decides not to continue the sentence

In [97]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 1

Or it just repeats the last questions, but first it writes "A:"

In [98]:
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 1

Let's try another example

In [112]:
text = df.loc[0, "CoT_directions"]

In [113]:
text

"Use mathemathical operations and write step by step the explanation of the following math problem solution:\nQ: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\nA: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.\nUse mathemathical operations and write step by step the explanation of the following math problem solution:\nQ: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\nHere is the solution explanation:"

We see, that model just repeats the question unfortunatelly

In [114]:
inputs = tokenizer.encode(text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128, num_return_sequences=4, num_beams=4)

print(tokenizer.decode(outputs[1], skip_special_tokens=True))
print()
print(tokenizer.decode(outputs[2], skip_special_tokens=True))

Use mathemathical operations and write step by step the explanation of the following math problem solution:
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Use mathemathical operations and write step by step the explanation of the following math problem solution:
Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Here is the solution explanation:
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She

Let's try paths prompts

In [124]:
text = df.loc[0, "CoT_paths"]
              
inputs = tokenizer.encode(text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128, num_return_sequences=4, num_beams=4)

print(tokenizer.decode(outputs[1], skip_special_tokens=True))
print()
print(tokenizer.decode(outputs[2], skip_special_tokens=True))

Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
A: In the beginning, Betty has only $100 / 2 = $50. Betty's grandparents gave her $15 * 2 = $30. This means, Betty needs 100 - 50 - 30 - 15 = $5 more. The answer is 5.
A: She had only half of the money she needs, that is, she needs $100 / 2 = $50. Her parents gave her $15. Grandparents gave her $15 * 2 = $30. This means, Betty needs 100 - 50 - 15 - 30 = $5 more. The answer is 5.
A: Betty had only half of the wallet cost, she had $100 / 2 = $50. Then grandparents gave her twice much as parents did, $15 * 2 = $30. Betty has 50 + 30 + 15 = $95. Wallet costs 100. She needs 100 - 95 = $5 more. The answer is 5.

Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every d

Sometimes model tries to count something, but it is absolutelly wrong and random

In [128]:
text = df.loc[1, "CoT_paths"]
              
inputs = tokenizer.encode(text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=128, num_return_sequences=4, num_beams=4)

print(tokenizer.decode(outputs[1], skip_special_tokens=True))
print()
print(tokenizer.decode(outputs[2], skip_special_tokens=True))

Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
A: In the beginning, Betty has only $100 / 2 = $50. Betty's grandparents gave her $15 * 2 = $30. This means, Betty needs 100 - 50 - 30 - 15 = $5 more. The answer is 5.
A: She had only half of the money she needs, that is, she needs $100 / 2 = $50. Her parents gave her $15. Grandparents gave her $15 * 2 = $30. This means, Betty needs 100 - 50 - 15 - 30 = $5 more. The answer is 5.
A: Betty had only half of the wallet cost, she had $100 / 2 = $50. Then grandparents gave her twice much as parents did, $15 * 2 = $30. Betty has 50 + 30 + 15 = $95. Wallet costs 100. She needs 100 - 95 = $5 more. The answer is 5.

Q: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take? What is the t